In [1]:
import importlib
import json
import os
from pathlib import Path
import sys

from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from dotenv import find_dotenv, load_dotenv
import pandas as pd
from modeling import Country, ModelingAccessor

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [2]:
load_dotenv(find_dotenv())

dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'

dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

In [3]:
bg_pth = gdb_raw/'block_group_patterns'

In [4]:
bg_df = pd.DataFrame.spatial.from_featureclass(bg_pth).drop(columns='OBJECTID')

print(bg_df.info())
bg_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   fips                    1148 non-null   object  
 1   season_2017_2018        1148 non-null   float64 
 2   season_2018_2019        1148 non-null   float64 
 3   season_2018_2019_delta  1148 non-null   float64 
 4   season_2019_2020        1148 non-null   float64 
 5   season_2019_2020_delta  1148 non-null   float64 
 6   season_2020_2021        1148 non-null   float64 
 7   season_2020_2021_delta  1148 non-null   float64 
 8   SHAPE                   1148 non-null   geometry
dtypes: float64(7), geometry(1), object(1)
memory usage: 80.8+ KB
None


,fips,season_2017_2018,season_2018_2019,season_2018_2019_delta,season_2019_2020,season_2019_2020_delta,season_2020_2021,season_2020_2021_delta,SHAPE
0,483396941011,0.0,4.0,4.0,0.0,-4.0,0.0,0.0,"{""rings"": [[[-95.47016593699999, 30.4217359920..."
1,481677212021,0.0,0.0,0.0,4.0,4.0,0.0,-4.0,"{""rings"": [[[-95.00105795299999, 29.5088950600..."
2,483396920022,0.0,0.0,0.0,4.0,4.0,0.0,-4.0,"{""rings"": [[[-95.35640506199996, 30.0768950010..."
3,150030099022,0.0,4.0,4.0,0.0,-4.0,0.0,0.0,"{""rings"": [[[-158.10524106999998, 21.579959060..."
4,150030101003,0.0,4.0,4.0,0.0,-4.0,0.0,0.0,"{""rings"": [[[-157.94991902799998, 21.692431970..."


In [5]:
usa = Country('USA', source='local')

usa

<modeling.Country - USA (local 2020)>

In [6]:
evars = usa.enrich_variables.drop_duplicates('name').reset_index(drop=True)

evars

,name,alias,data_collection,enrich_name,enrich_field_name
0,AGE0_CY,2020 Population Age <1,1yearincrements,1yearincrements.AGE0_CY,F1yearincrements_AGE0_CY
1,AGE1_CY,2020 Population Age 1,1yearincrements,1yearincrements.AGE1_CY,F1yearincrements_AGE1_CY
2,AGE2_CY,2020 Population Age 2,1yearincrements,1yearincrements.AGE2_CY,F1yearincrements_AGE2_CY
3,AGE3_CY,2020 Population Age 3,1yearincrements,1yearincrements.AGE3_CY,F1yearincrements_AGE3_CY
4,AGE4_CY,2020 Population Age 4,1yearincrements,1yearincrements.AGE4_CY,F1yearincrements_AGE4_CY
...,...,...,...,...,...
8271,ACSRMV2000,2018 RHHs/Moved In: 2000-2009 (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV2000,yearmovedin_ACSRMV2000
8272,ACSRMV1990,2018 RHHs/Moved In: 1990-1999 (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV1990,yearmovedin_ACSRMV1990
8273,ACSRMV1989,2018 RHHs/Moved In: 1989/Before (ACS 5-Yr),yearmovedin,yearmovedin.ACSRMV1989,yearmovedin_ACSRMV1989
8274,ACSMEDYRMV,2018 Median Year Householder Moved In (ACS 5-Yr),yearmovedin,yearmovedin.ACSMEDYRMV,yearmovedin_ACSMEDYRMV


In [7]:
gis = GIS(os.getenv('BA_QA_URL'), username=os.getenv('BA_QA_USERNAME'), password=os.getenv('BA_QA_PASSWORD'))
usa_gis = Country('USA', source=gis)

usa_gis

<modeling.Country - USA (GIS at https://baqa.mapsqa.arcgis.com logged in as jmccune_baqa)>

In [8]:
usa.geography_levels

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
9,states,States,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...


In [10]:
bg_enrich_df = bg_df.mdl.enrich(evars, enrich_geography_level='block_groups', enrich_id_column='fips', source=usa)

print(bg_enrich_df.info())
bg_enrich_df.head()

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
Failed to execute (EnrichLayer).
